In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25359478
paper_name = 'jayakody_kitagaki_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Final results 1 fileLahiruscreening.xls', sheet_name='All screen data', skiprows=8)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5139 x 6


In [7]:
original_data.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Mutant ', 'SI AVG', 'SEM', 'Tscore'], dtype='object')

In [8]:
original_data['orfs'] = original_data['Mutant '].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orfs'] = clean_orf(original_data['orfs'])

In [10]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['orfs'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, Mutant , SI AVG, SEM, Tscore, orfs]
Index: []


In [12]:
original_data['data'] = original_data['SI AVG']

In [13]:
original_data.set_index('orfs', inplace=True)

In [14]:
original_data.index.name='orf'

In [15]:
original_data['data'] = pd.to_numeric(original_data['data'], errors='coerce', downcast='float')

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(4301, 2)

# Prepare the final dataset

In [25]:
data = original_data[['data']].copy()

In [26]:
dataset_ids = [16610]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
data.head()

dataset_id,16610
data_type,value
orf,
YAL002W,0.957117
YAL004W,1.018766
YAL005C,0.980901
YAL007C,1.061417
YAL008W,0.994281


## Subset to the genes currently in SGD

In [29]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [30]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16610
,data_type,value
gene_id,orf,
2,YAL002W,0.957117
1863,YAL004W,1.018766
4,YAL005C,0.980901
5,YAL007C,1.061417
6,YAL008W,0.994281


# Normalize

In [32]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [33]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [34]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16610          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.957117 -0.413429
1863    YAL004W  1.018766 -0.084383
4       YAL005C  0.980901 -0.286482
5       YAL007C  1.061417  0.143262
6       YAL008W  0.994281 -0.215068

# Print out

In [35]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 25359478...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.84s/it]

Updating the data_modified_on field...
